In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, EfficientNetB0
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os

In [2]:
# Dataset Path
dataset_path = '/kaggle/input/dog-vs-cat/animals'  # Change if running locally
img_size = (150, 150)  # Increased for better feature extraction
batch_size = 32

In [3]:
# Data Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [4]:
train_data = datagen.flow_from_directory(dataset_path, target_size=img_size, batch_size=batch_size, class_mode='binary', subset='training')
val_data = datagen.flow_from_directory(dataset_path, target_size=img_size, batch_size=batch_size, class_mode='binary', subset='validation')

Found 800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [5]:
base_model = VGG16(weights=None, include_top=False, input_shape=(150,150,3))

In [6]:
# Custom Head for Fine-Tuning
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

In [7]:
# Compile Model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [9]:
model.fit(train_data, validation_data=val_data, epochs=5, callbacks=[early_stop])

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 417s 16s/step - accuracy: 0.4735 - loss: 0.6934 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 397s 15s/step - accuracy: 0.5043 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 369s 15s/step - accuracy: 0.4789 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 369s 15s/step - accuracy: 0.4977 - loss: 0.6931 - val_accuracy: 0.5000 - val_loss: 0.6915
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 385s 15s/step - accuracy: 0.5150 - loss: 0.6745 - val_accuracy: 0.6650 - val_loss: 0.6325


In [10]:
model.save("/kaggle/working/vgg16_dog_vs_cat.h5")